In [1]:
from google.colab import drive
drive.mount('/content/drive')
print('mount success')

Mounted at /content/drive
mount success


In [2]:
import os
import json
import pandas as pd

##Data

In [3]:
# test ground-truth captions
test_captions_gt_path = '/content/drive/Shareddrives/msvd-dataset/test/test_video_captions_preprocessed.json'

In [48]:
# test generated captions
test_captions_gen_path = '/content/drive/Shareddrives/msvd-dataset/test/test_video_captions_generated_Trained_model_greedy.txt'

In [5]:
def json_to_dataframe(json_file):
  # Read data from the JSON file
  with open(json_file, "r") as f:
    data = json.load(f)

  # Create a list to store data for the DataFrame
  data_list = []
  for entry in data:
    video_id = entry["id"]
    ground_truths = entry["caption"]
    data_list.append({"video_id": video_id, "ground_truths": ground_truths})

  # Create the DataFrame
  df = pd.DataFrame(data_list)

  return df

In [6]:
def update_dataframe(df, captions_file):
  # Read data from the text file into a DataFrame with column names
  captions_df = pd.read_csv(captions_file, delimiter=",", names=["video_id", "generated_caption", "time_taken"])

  def add_avi_extension(video_id):
    """
    Adds '.avi' extension to the video ID.
    """
    return f"{video_id}.avi"

  # Apply function to add '.avi' extension to video_id in captions_df
  captions_df["video_id"] = captions_df["video_id"].apply(add_avi_extension)

  # Merge DataFrames based on video_id (assuming video_id is unique in both)
  df = df.merge(captions_df[["video_id", "generated_caption", "time_taken"]], how="left", on="video_id")

  print("DataFrame updated with generated captions and time taken")
  return df

In [7]:
def get_captions(df, ground_truth_col="ground_truths", generated_caption_col="generated_caption"):
  references = df[ground_truth_col].tolist()
  candidates = df[generated_caption_col].tolist()

  return references, candidates

In [49]:
df = json_to_dataframe(test_captions_gt_path)

In [50]:
df = update_dataframe(df, test_captions_gen_path)

DataFrame updated with generated captions and time taken


In [51]:
df

,video_id,ground_truths,generated_caption,time_taken
0,eyhzdC936uk_15_27.avi,"[a person plays high five with his small dog, ...",a woman is looking out from a,1.88
1,f-24IxG9ijw_25_40.avi,[a child is riding a motorcycle on the seashor...,a woman is riding on a,0.95
2,f9Won2JpOEU_60_80.avi,"[a cat is laying down licking it s paw, a cat ...",a woman is jumping at a,1.25
3,f9_bP219ehQ_63_70.avi,"[a person being interviewed at a park event, a...",a man is eating on a,1.34
4,fBA_lxUiwSg_2_4.avi,"[a cat is jumping on the table, a cat jumps fr...",two men are walking,1.10
...,...,...,...,...
665,zulPFoY64wE_26_33.avi,"[a person is riding a horse, a person is ridin...",a man is running on the ground,1.22
666,zv2RIbUsnSw_159_162.avi,"[children are sliding on black tarp, two kids ...",a man is taking a on of,1.22
667,zv2RIbUsnSw_335_341.avi,"[a person is being rolled on a gurney, a perso...",a man is spinning on a,1.10
668,zxB4dFJhHR8_1_9.avi,"[a child riding a bicycle along the beach, a b...",a man is riding a bike,1.13


In [52]:
df = df.dropna()

In [53]:
df

,video_id,ground_truths,generated_caption,time_taken
0,eyhzdC936uk_15_27.avi,"[a person plays high five with his small dog, ...",a woman is looking out from a,1.88
1,f-24IxG9ijw_25_40.avi,[a child is riding a motorcycle on the seashor...,a woman is riding on a,0.95
2,f9Won2JpOEU_60_80.avi,"[a cat is laying down licking it s paw, a cat ...",a woman is jumping at a,1.25
3,f9_bP219ehQ_63_70.avi,"[a person being interviewed at a park event, a...",a man is eating on a,1.34
4,fBA_lxUiwSg_2_4.avi,"[a cat is jumping on the table, a cat jumps fr...",two men are walking,1.10
...,...,...,...,...
665,zulPFoY64wE_26_33.avi,"[a person is riding a horse, a person is ridin...",a man is running on the ground,1.22
666,zv2RIbUsnSw_159_162.avi,"[children are sliding on black tarp, two kids ...",a man is taking a on of,1.22
667,zv2RIbUsnSw_335_341.avi,"[a person is being rolled on a gurney, a perso...",a man is spinning on a,1.10
668,zxB4dFJhHR8_1_9.avi,"[a child riding a bicycle along the beach, a b...",a man is riding a bike,1.13


In [54]:
references, candidates = get_captions(df)

print(f"Number of references: {len(references)}")
print(f"Number of candidates: {len(candidates)}")

Number of references: 670
Number of candidates: 670


#Evaluation

In [55]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
def get_average_score(df, score_column):
  # Check if the column exists
  if score_column not in df.columns:
    print(f"Column '{score_column}' not found in the DataFrame")

  # Calculate the average score
  average_score = df[score_column].mean()

  return average_score

##Time taken

In [57]:
avg_time_taken = round(get_average_score(df, "time_taken"),2)

In [58]:
avg_time_taken

1.18

##BLEU

###Corpus Bleu

In [66]:
from nltk.translate.bleu_score import corpus_bleu

In [67]:
bleu4_score = round(corpus_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25)),2)

In [68]:
bleu4_score

0.57

##METEOR

In [69]:
from nltk.translate.meteor_score import meteor_score

In [70]:
def _calculate_meteor_per_row(ground_truths, generated_caption):
  """
  Calculates BLEU-4 score for a single video entry.
  """
  reference = [caption.split() for caption in ground_truths]
  candidate = generated_caption.split()
  score = nltk.translate.meteor_score.meteor_score(reference, candidate)
  return round(score,2)

In [71]:
def calculate_meteor_score(df, ground_truth_col="ground_truths", generated_caption_col="generated_caption"):
  df["METEOR"] = df.apply(lambda row: _calculate_meteor_per_row(row[ground_truth_col], row[generated_caption_col]), axis=1)
  return df

In [72]:
df = calculate_meteor_score(df)

In [73]:
df

,video_id,ground_truths,generated_caption,time_taken,BLEU-4,METEOR
0,eyhzdC936uk_15_27.avi,"[a person plays high five with his small dog, ...",a woman is looking out from a,1.88,0.00,0.21
1,f-24IxG9ijw_25_40.avi,[a child is riding a motorcycle on the seashor...,a woman is riding on a,0.95,0.00,0.53
2,f9Won2JpOEU_60_80.avi,"[a cat is laying down licking it s paw, a cat ...",a woman is jumping at a,1.25,0.00,0.17
3,f9_bP219ehQ_63_70.avi,"[a person being interviewed at a park event, a...",a man is eating on a,1.34,0.00,0.46
4,fBA_lxUiwSg_2_4.avi,"[a cat is jumping on the table, a cat jumps fr...",two men are walking,1.10,0.00,0.00
...,...,...,...,...,...,...
665,zulPFoY64wE_26_33.avi,"[a person is riding a horse, a person is ridin...",a man is running on the ground,1.22,0.00,0.25
666,zv2RIbUsnSw_159_162.avi,"[children are sliding on black tarp, two kids ...",a man is taking a on of,1.22,0.00,0.14
667,zv2RIbUsnSw_335_341.avi,"[a person is being rolled on a gurney, a perso...",a man is spinning on a,1.10,0.00,0.40
668,zxB4dFJhHR8_1_9.avi,"[a child riding a bicycle along the beach, a b...",a man is riding a bike,1.13,0.54,0.81


In [74]:
meteor_score = round(get_average_score(df, "METEOR"),2)

In [75]:
meteor_score

0.37

##ROUGE

In [76]:
!pip install evaluate
!pip install rouge-score

In [77]:
import evaluate
rouge = evaluate.load('rouge')

In [78]:
results = rouge.compute(predictions=candidates, references=references)
print(results)

{'rouge1': 0.5213036657275028, 'rouge2': 0.1570958146331285, 'rougeL': 0.5180989423219456, 'rougeLsum': 0.5181139633949119}


In [79]:
rougeL_score = round(results['rougeL'],2)

In [80]:
rougeL_score

0.52

##CIDEr

In [81]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
class CIDEr:
    def __init__(self, references):
        self.references = references
        self.vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
        self.reference_vectors = self.vectorizer.fit_transform(references)

    def compute_cider(self, hypothesis):
        hypothesis_vector = self.vectorizer.transform([hypothesis])
        cider_score = cosine_similarity(self.reference_vectors, hypothesis_vector).mean()
        return cider_score

In [83]:
def _calculate_cider_per_row(ground_truths, generated_caption):
  """
  Calculates BLEU-4 score for a single video entry.
  """
  reference = ground_truths
  candidate = generated_caption

  cider = CIDEr(reference)
  score = cider.compute_cider(candidate)
  return round(score,2)

In [84]:
def calculate_cider_score(df, ground_truth_col="ground_truths", generated_caption_col="generated_caption"):
  df["CIDER"] = df.apply(lambda row: _calculate_cider_per_row(row[ground_truth_col], row[generated_caption_col]), axis=1)
  return df

In [85]:
df = calculate_cider_score(df)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [86]:
df

,video_id,ground_truths,generated_caption,time_taken,BLEU-4,METEOR,CIDER
0,eyhzdC936uk_15_27.avi,"[a person plays high five with his small dog, ...",a woman is looking out from a,1.88,0.00,0.21,0.29
1,f-24IxG9ijw_25_40.avi,[a child is riding a motorcycle on the seashor...,a woman is riding on a,0.95,0.00,0.53,0.44
2,f9Won2JpOEU_60_80.avi,"[a cat is laying down licking it s paw, a cat ...",a woman is jumping at a,1.25,0.00,0.17,0.23
3,f9_bP219ehQ_63_70.avi,"[a person being interviewed at a park event, a...",a man is eating on a,1.34,0.00,0.46,0.23
4,fBA_lxUiwSg_2_4.avi,"[a cat is jumping on the table, a cat jumps fr...",two men are walking,1.10,0.00,0.00,0.00
...,...,...,...,...,...,...,...
665,zulPFoY64wE_26_33.avi,"[a person is riding a horse, a person is ridin...",a man is running on the ground,1.22,0.00,0.25,0.36
666,zv2RIbUsnSw_159_162.avi,"[children are sliding on black tarp, two kids ...",a man is taking a on of,1.22,0.00,0.14,0.15
667,zv2RIbUsnSw_335_341.avi,"[a person is being rolled on a gurney, a perso...",a man is spinning on a,1.10,0.00,0.40,0.39
668,zxB4dFJhHR8_1_9.avi,"[a child riding a bicycle along the beach, a b...",a man is riding a bike,1.13,0.54,0.81,0.47


In [87]:
cider_score = round(get_average_score(df, "CIDER"),2)

In [88]:
cider_score

0.3